<a href="https://colab.research.google.com/github/kunaldash/Data-Science/blob/master/Analysis_of_eCommerce_data_GLTC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Client Requirements:
1. Find the top spenders or spending customers
2. Identify loyal customers
3. Find repeat customers
4. Which products sell together?

Install necessary packages as required.

In [ ]:
!pip install scikit-learn

In [ ]:
!pip install tensorflow

In [ ]:
!pip install seaborn

In [ ]:
!pip install pickle-mixin

  Created wheel for pickle-mixin: filename=pickle_mixin-1.0.2-cp36-none-any.whl size=6000 sha256=aea6fae83c50990e64d4fbf25e17ddf6d23f1e533ee00cd3bc6b5cd4493b782f
  Stored in directory: /root/.cache/pip/wheels/cd/05/42/71de70fa36b9cbb7657bb5793a16f8028c1cdc1bdd3b8e1ac3
Successfully built pickle-mixin


Import necessary libraries.

In [ ]:
import numpy as np
import pandas as pd
import scipy
import seaborn as sns
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


The first objective of the retailer is to segment the retailer using a couple of features.
Therefore, as the first step, read in the data:

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving GLTC Data.xlsx to GLTC Data.xlsx


Next read the imported data file

In [ ]:
import io
df = pd.read_excel(io.BytesIO(uploaded['GLTC Data.xlsx']))

Explore the dataset

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21840 entries, 0 to 21839
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Date                   21840 non-null  datetime64[ns]
 1   Time                   21840 non-null  object        
 2   Time Zone              21840 non-null  object        
 3   Category               21840 non-null  object        
 4   Item                   21840 non-null  object        
 5   Quantity               21840 non-null  int64         
 6   Price Point Name       15670 non-null  object        
 7   SKU                    8324 non-null   object        
 8   Modifiers Applied      21840 non-null  object        
 9   Gross Sales            21840 non-null  float64       
 10  Discounts              21840 non-null  float64       
 11  Net Sales              21840 non-null  float64       
 12  Tax                    21840 non-null  float64       
 13  T

In [ ]:
df.head()

,Date,Time,Time Zone,Category,Sales type,Item,Quantity,Price Point Name,SKU,Modifiers Applied,Gross Sales,Discounts,Net Sales,Tax,Transaction ID,Device Name,Event Type,Location,Dining Option,Customer ID,Customer Reference ID,Unit,Count
0,2017-08-30,11:11:23,Central Time (US & Canada),Loose tea,NaN,"Green Flower Rooibos, organic",1,4oz,greenflowerrooibos-4,NaN,11.50,0.0,11.50,0.00,AxVUvEsgClKYFnYsEPqwZsveV,iPad,Payment,Green Leaf Tea Co.,For Here,0078JQACVD65AKGBXWR43K5C6W,NaN,ea,1
1,2017-08-30,11:11:23,Central Time (US & Canada),Drinks,NaN,Iced Tea,1,Regular,drink-icedtea,NaN,2.70,0.0,2.70,0.25,AxVUvEsgClKYFnYsEPqwZsveV,iPad,Payment,Green Leaf Tea Co.,For Here,0078JQACVD65AKGBXWR43K5C6W,NaN,ea,1
2,2017-12-09,10:35:42,Central Time (US & Canada),Loose tea,NaN,Energizing Harmony,1,2 oz,NaN,NaN,9.00,-1.8,7.20,0.00,6GNSV79jmSWjWO6rCx38oBmeV,iPad,Payment,Green Leaf Tea Co.,For Here,00BHWA7X7D3A0GDA9N2YP5ES2C,NaN,ea,1
3,2017-12-09,10:35:42,Central Time (US & Canada),Infusers,NaN,Basket Infuser,1,Regular,basketinfuser,NaN,9.75,0.0,9.75,0.71,6GNSV79jmSWjWO6rCx38oBmeV,iPad,Payment,Green Leaf Tea Co.,For Here,00BHWA7X7D3A0GDA9N2YP5ES2C,NaN,ea,1
4,2017-12-09,10:35:42,Central Time (US & Canada),Drinks,NaN,Hot Tea,1,Regular,drink-hottea,NaN,2.70,0.0,2.70,0.24,6GNSV79jmSWjWO6rCx38oBmeV,iPad,Payment,Green Leaf Tea Co.,For Here,00BHWA7X7D3A0GDA9N2YP5ES2C,NaN,ea,1


In [ ]:
df.tail()

,Date,Time,Time Zone,Category,Sales type,Item,Quantity,Price Point Name,SKU,Modifiers Applied,Gross Sales,Discounts,Net Sales,Tax,Transaction ID,Device Name,Event Type,Location,Dining Option,Customer ID,Customer Reference ID,Unit,Count
21835,2017-01-12,12:33:15,Central Time (US & Canada),Loose tea,NaN,Pu-erh Ginger,1,4oz,NaN,NaN,34.45,-7.0,27.45,2.54,8ZUnU3hM52Hguz5EaJXaz27eV,NaN,Payment,Green Leaf Tea Co.,For Here,NaN,NaN,ea,1
21836,2017-01-12,12:33:15,Central Time (US & Canada),Infusers,NaN,Basket Infuser,1,NaN,NaN,NaN,9.75,0.0,9.75,0.71,8ZUnU3hM52Hguz5EaJXaz27eV,NaN,Payment,Green Leaf Tea Co.,For Here,NaN,NaN,ea,1
21837,2017-01-12,11:54:15,Central Time (US & Canada),Chocolate- Tea Room,NaN,Mayan Pepper Chai,1,NaN,NaN,NaN,3.75,0.0,3.75,0.27,kDlgDRuO8JpcJL5bbkw1Ny8eV,NaN,Payment,Green Leaf Tea Co.,For Here,NaN,NaN,ea,1
21838,2017-01-12,11:54:15,Central Time (US & Canada),Chocolate- Tea Room,NaN,Earl Grey Green,1,NaN,NaN,NaN,3.75,0.0,3.75,0.27,kDlgDRuO8JpcJL5bbkw1Ny8eV,NaN,Payment,Green Leaf Tea Co.,For Here,NaN,NaN,ea,1
21839,2017-01-12,11:54:15,Central Time (US & Canada),Loose tea,NaN,Citrus Eucalyptus,1,2oz,NaN,NaN,4.50,0.0,4.50,0.33,kDlgDRuO8JpcJL5bbkw1Ny8eV,NaN,Payment,Green Leaf Tea Co.,For Here,NaN,NaN,ea,1


Drop the "Sales Type" feature as it has no data in it.

In [ ]:
df.drop(['Sales type'], axis=1)

In [ ]:
df.head()

,Date,Time,Time Zone,Category,Item,Quantity,Price Point Name,SKU,Modifiers Applied,Gross Sales,Discounts,Net Sales,Tax,Transaction ID,Device Name,Event Type,Location,Dining Option,Customer ID,Customer Reference ID,Unit,Count
0,2017-08-30,11:11:23,Central Time (US & Canada),Loose tea,"Green Flower Rooibos, organic",1,4oz,greenflowerrooibos-4,No,11.50,0.0,11.50,0.00,AxVUvEsgClKYFnYsEPqwZsveV,iPad,Payment,Green Leaf Tea Co.,For Here,0078JQACVD65AKGBXWR43K5C6W,NaN,ea,1
1,2017-08-30,11:11:23,Central Time (US & Canada),Drinks,Iced Tea,1,Regular,drink-icedtea,No,2.70,0.0,2.70,0.25,AxVUvEsgClKYFnYsEPqwZsveV,iPad,Payment,Green Leaf Tea Co.,For Here,0078JQACVD65AKGBXWR43K5C6W,NaN,ea,1
2,2017-12-09,10:35:42,Central Time (US & Canada),Loose tea,Energizing Harmony,1,2 oz,NaN,No,9.00,-1.8,7.20,0.00,6GNSV79jmSWjWO6rCx38oBmeV,iPad,Payment,Green Leaf Tea Co.,For Here,00BHWA7X7D3A0GDA9N2YP5ES2C,NaN,ea,1
3,2017-12-09,10:35:42,Central Time (US & Canada),Infusers,Basket Infuser,1,Regular,basketinfuser,No,9.75,0.0,9.75,0.71,6GNSV79jmSWjWO6rCx38oBmeV,iPad,Payment,Green Leaf Tea Co.,For Here,00BHWA7X7D3A0GDA9N2YP5ES2C,NaN,ea,1
4,2017-12-09,10:35:42,Central Time (US & Canada),Drinks,Hot Tea,1,Regular,drink-hottea,No,2.70,0.0,2.70,0.24,6GNSV79jmSWjWO6rCx38oBmeV,iPad,Payment,Green Leaf Tea Co.,For Here,00BHWA7X7D3A0GDA9N2YP5ES2C,NaN,ea,1


Replace the "NaN" value in "Modifiers Applied" feature to "No".

In [ ]:
df["Modifiers Applied"].fillna("No", inplace = True)
df

,Date,Time,Time Zone,Category,Item,Quantity,Price Point Name,SKU,Modifiers Applied,Gross Sales,Discounts,Net Sales,Tax,Transaction ID,Device Name,Event Type,Location,Dining Option,Customer ID,Customer Reference ID,Unit,Count
0,2017-08-30,11:11:23,Central Time (US & Canada),Loose tea,"Green Flower Rooibos, organic",1,4oz,greenflowerrooibos-4,No,11.50,0.0,11.50,0.00,AxVUvEsgClKYFnYsEPqwZsveV,iPad,Payment,Green Leaf Tea Co.,For Here,0078JQACVD65AKGBXWR43K5C6W,NaN,ea,1
1,2017-08-30,11:11:23,Central Time (US & Canada),Drinks,Iced Tea,1,Regular,drink-icedtea,No,2.70,0.0,2.70,0.25,AxVUvEsgClKYFnYsEPqwZsveV,iPad,Payment,Green Leaf Tea Co.,For Here,0078JQACVD65AKGBXWR43K5C6W,NaN,ea,1
2,2017-12-09,10:35:42,Central Time (US & Canada),Loose tea,Energizing Harmony,1,2 oz,NaN,No,9.00,-1.8,7.20,0.00,6GNSV79jmSWjWO6rCx38oBmeV,iPad,Payment,Green Leaf Tea Co.,For Here,00BHWA7X7D3A0GDA9N2YP5ES2C,NaN,ea,1
3,2017-12-09,10:35:42,Central Time (US & Canada),Infusers,Basket Infuser,1,Regular,basketinfuser,No,9.75,0.0,9.75,0.71,6GNSV79jmSWjWO6rCx38oBmeV,iPad,Payment,Green Leaf Tea Co.,For Here,00BHWA7X7D3A0GDA9N2YP5ES2C,NaN,ea,1
4,2017-12-09,10:35:42,Central Time (US & Canada),Drinks,Hot Tea,1,Regular,drink-hottea,No,2.70,0.0,2.70,0.24,6GNSV79jmSWjWO6rCx38oBmeV,iPad,Payment,Green Leaf Tea Co.,For Here,00BHWA7X7D3A0GDA9N2YP5ES2C,NaN,ea,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21835,2017-01-12,12:33:15,Central Time (US & Canada),Loose tea,Pu-erh Ginger,1,4oz,NaN,No,34.45,-7.0,27.45,2.54,8ZUnU3hM52Hguz5EaJXaz27eV,NaN,Payment,Green Leaf Tea Co.,For Here,NaN,NaN,ea,1
21836,2017-01-12,12:33:15,Central Time (US & Canada),Infusers,Basket Infuser,1,NaN,NaN,No,9.75,0.0,9.75,0.71,8ZUnU3hM52Hguz5EaJXaz27eV,NaN,Payment,Green Leaf Tea Co.,For Here,NaN,NaN,ea,1
21837,2017-01-12,11:54:15,Central Time (US & Canada),Chocolate- Tea Room,Mayan Pepper Chai,1,NaN,NaN,No,3.75,0.0,3.75,0.27,kDlgDRuO8JpcJL5bbkw1Ny8eV,NaN,Payment,Green Leaf Tea Co.,For Here,NaN,NaN,ea,1
21838,2017-01-12,11:54:15,Central Time (US & Canada),Chocolate- Tea Room,Earl Grey Green,1,NaN,NaN,No,3.75,0.0,3.75,0.27,kDlgDRuO8JpcJL5bbkw1Ny8eV,NaN,Payment,Green Leaf Tea Co.,For Here,NaN,NaN,ea,1


Replace "NaN" values in other features throughout the dataset as applicable.

In [ ]:
df["Price Point Name"].fillna("Not available", inplace = True)
df["SKU"].fillna("Not available", inplace = True)
df.head()

,Date,Time,Time Zone,Category,Item,Quantity,Price Point Name,SKU,Modifiers Applied,Gross Sales,Discounts,Net Sales,Tax,Transaction ID,Device Name,Event Type,Location,Dining Option,Customer ID,Customer Reference ID,Unit,Count
0,2017-08-30,11:11:23,Central Time (US & Canada),Loose tea,"Green Flower Rooibos, organic",1,4oz,greenflowerrooibos-4,No,11.50,0.0,11.50,0.00,AxVUvEsgClKYFnYsEPqwZsveV,iPad,Payment,Green Leaf Tea Co.,For Here,0078JQACVD65AKGBXWR43K5C6W,NaN,ea,1
1,2017-08-30,11:11:23,Central Time (US & Canada),Drinks,Iced Tea,1,Regular,drink-icedtea,No,2.70,0.0,2.70,0.25,AxVUvEsgClKYFnYsEPqwZsveV,iPad,Payment,Green Leaf Tea Co.,For Here,0078JQACVD65AKGBXWR43K5C6W,NaN,ea,1
2,2017-12-09,10:35:42,Central Time (US & Canada),Loose tea,Energizing Harmony,1,2 oz,Not available,No,9.00,-1.8,7.20,0.00,6GNSV79jmSWjWO6rCx38oBmeV,iPad,Payment,Green Leaf Tea Co.,For Here,00BHWA7X7D3A0GDA9N2YP5ES2C,NaN,ea,1
3,2017-12-09,10:35:42,Central Time (US & Canada),Infusers,Basket Infuser,1,Regular,basketinfuser,No,9.75,0.0,9.75,0.71,6GNSV79jmSWjWO6rCx38oBmeV,iPad,Payment,Green Leaf Tea Co.,For Here,00BHWA7X7D3A0GDA9N2YP5ES2C,NaN,ea,1
4,2017-12-09,10:35:42,Central Time (US & Canada),Drinks,Hot Tea,1,Regular,drink-hottea,No,2.70,0.0,2.70,0.24,6GNSV79jmSWjWO6rCx38oBmeV,iPad,Payment,Green Leaf Tea Co.,For Here,00BHWA7X7D3A0GDA9N2YP5ES2C,NaN,ea,1


Generate month and year from the Date feature

In [ ]:
df['year'] = pd.DatetimeIndex(df['Date']).year
df['month'] = pd.DatetimeIndex(df['Date']).month
df.head()

,Date,Time,Time Zone,Category,Item,Quantity,Price Point Name,SKU,Modifiers Applied,Gross Sales,Discounts,Net Sales,Tax,Transaction ID,Device Name,Event Type,Location,Dining Option,Customer ID,Customer Reference ID,Unit,Count,year,month
0,2017-08-30,11:11:23,Central Time (US & Canada),Loose tea,"Green Flower Rooibos, organic",1,4oz,greenflowerrooibos-4,No,11.50,0.0,11.50,0.00,AxVUvEsgClKYFnYsEPqwZsveV,iPad,Payment,Green Leaf Tea Co.,For Here,0078JQACVD65AKGBXWR43K5C6W,NaN,ea,1,2017,8
1,2017-08-30,11:11:23,Central Time (US & Canada),Drinks,Iced Tea,1,Regular,drink-icedtea,No,2.70,0.0,2.70,0.25,AxVUvEsgClKYFnYsEPqwZsveV,iPad,Payment,Green Leaf Tea Co.,For Here,0078JQACVD65AKGBXWR43K5C6W,NaN,ea,1,2017,8
2,2017-12-09,10:35:42,Central Time (US & Canada),Loose tea,Energizing Harmony,1,2 oz,Not available,No,9.00,-1.8,7.20,0.00,6GNSV79jmSWjWO6rCx38oBmeV,iPad,Payment,Green Leaf Tea Co.,For Here,00BHWA7X7D3A0GDA9N2YP5ES2C,NaN,ea,1,2017,12
3,2017-12-09,10:35:42,Central Time (US & Canada),Infusers,Basket Infuser,1,Regular,basketinfuser,No,9.75,0.0,9.75,0.71,6GNSV79jmSWjWO6rCx38oBmeV,iPad,Payment,Green Leaf Tea Co.,For Here,00BHWA7X7D3A0GDA9N2YP5ES2C,NaN,ea,1,2017,12
4,2017-12-09,10:35:42,Central Time (US & Canada),Drinks,Hot Tea,1,Regular,drink-hottea,No,2.70,0.0,2.70,0.24,6GNSV79jmSWjWO6rCx38oBmeV,iPad,Payment,Green Leaf Tea Co.,For Here,00BHWA7X7D3A0GDA9N2YP5ES2C,NaN,ea,1,2017,12


Which unique items were sold over time?

In [ ]:
df["Item"].unique()

array(['Green Flower Rooibos, organic', 'Iced Tea', 'Energizing Harmony',
       'Basket Infuser', 'Hot Tea', 'Bubble Tea', 'Chai',
       'Dark Chocolate Mint Pu-erh', 'Citrus Green Dragon Tea',
       'Fruit Fiesta', 'Raspberry Leaf', 'Tea Cake',
       'Paper Tea Filter With Drawstring', 'Calming Harmony',
       'Rooibos Chai', 'Mandarin Silk', 'Earl Grey, Organic',
       'ACS 4 Piece Box', 'Custom Amount', 'Raspberry Rooibos',
       'AnXi Oolong', 'Honey Bush, organic', 'Rooibos Creamsicle',
       'Almond Apple', 'Cranberry Creme', 'Sweet Pomegranate',
       'Turmeric Chai, Organic', 'South of the Border',
       'ChaCult Basket Infuser', 'Decaf Ceylon', 'Finum Tea Filter',
       'Ginger Pu-Erh, organic', 'Restoring Harmony',
       'Matcha Bowl, Black', 'Chocolate Bar', 'Blueberry, Organic ',
       'Kukicha, organic', 'Silver Needles, Organic ',
       'Dragonwell, Organic', 'Eternal Peace', 'Kombucha', 'Shirt',
       'Winter Fair', 'Genmaicha, organic', 'Matcha Set',
    

Subset data by year

In [ ]:
df2019 = df[df.year == 2019]
df2019.head()

,Date,Time,Time Zone,Category,Item,Quantity,Price Point Name,SKU,Modifiers Applied,Gross Sales,Discounts,Net Sales,Tax,Transaction ID,Device Name,Event Type,Location,Dining Option,Customer ID,Customer Reference ID,Unit,Count,year,month
5,2019-03-07,15:36:36,Central Time (US & Canada),Drinks,Bubble Tea,1,Not available,Not available,No,4.00,0.0,4.00,0.36,uE7ApTzYadxHknth9Q4deN0eV,iPad,Payment,Green Leaf Tea Co.,For Here,00CDETBM9S7YCQSWJ00EY516T8,NaN,ea,1,2019,3
6,2019-01-10,14:06:16,Central Time (US & Canada),Drinks,Chai,1,Not available,Not available,No,3.00,0.0,3.00,0.27,8ho3ySW5YxH8CnCwgyALA4reV,iPad,Payment,Green Leaf Tea Co.,For Here,00CDETBM9S7YCQSWJ00EY516T8,NaN,ea,1,2019,1
27,2019-01-22,16:38:17,Central Time (US & Canada),Drinks,Bubble Tea,1,Not available,Not available,Large Size,5.25,0.0,5.25,0.47,sThBEkuJANY97s2gPJUp094eV,iPad,Payment,Green Leaf Tea Co.,For Here,014RGMMYD55FPSEEZSVSV0GNA4,NaN,ea,1,2019,1
42,2019-09-03,11:43:07,Central Time (US & Canada),Infusers,ChaCult Basket Infuser,1,Regular,Not available,No,8.75,0.0,8.75,0.62,AnK1tmyDKqXULRAuQyZNcA9eV,iPad,Payment,Green Leaf Tea Co.,For Here,04RWQ8129H48HC7DH9XE3VK3Q4,NaN,ea,1,2019,9
43,2019-09-03,11:43:07,Central Time (US & Canada),Loose tea,Cranberry Creme,1,4oz,cranberrycreme-4,No,10.50,0.0,10.50,0.00,AnK1tmyDKqXULRAuQyZNcA9eV,iPad,Payment,Green Leaf Tea Co.,For Here,04RWQ8129H48HC7DH9XE3VK3Q4,NaN,ea,1,2019,9


In [ ]:
df2018 = df[df.year == 2018]
df2018.head()

,Date,Time,Time Zone,Category,Item,Quantity,Price Point Name,SKU,Modifiers Applied,Gross Sales,Discounts,Net Sales,Tax,Transaction ID,Device Name,Event Type,Location,Dining Option,Customer ID,Customer Reference ID,Unit,Count,year,month
7,2018-12-19,14:00:55,Central Time (US & Canada),Drinks,Hot Tea,1,Regular,drink-hottea,No,2.70,0.0,2.70,0.24,6K5758xAuqjJINfx8jMsPOleV,iPad,Payment,Green Leaf Tea Co.,For Here,00CDETBM9S7YCQSWJ00EY516T8,NaN,ea,1,2018,12
8,2018-12-14,14:45:05,Central Time (US & Canada),Drinks,Bubble Tea,1,Not available,Not available,Large Size,5.15,0.0,5.15,0.46,kRk2WEc602fZs6nPTSkY2lxeV,iPad,Payment,Green Leaf Tea Co.,For Here,00CDETBM9S7YCQSWJ00EY516T8,NaN,ea,1,2018,12
9,2018-12-14,14:45:05,Central Time (US & Canada),Loose tea,Dark Chocolate Mint Pu-erh,1,sample,Not available,No,3.00,0.0,3.00,0.00,kRk2WEc602fZs6nPTSkY2lxeV,iPad,Payment,Green Leaf Tea Co.,For Here,00CDETBM9S7YCQSWJ00EY516T8,NaN,ea,1,2018,12
10,2018-12-14,14:45:05,Central Time (US & Canada),Loose tea,Citrus Green Dragon Tea,1,Sample,greencitrusdragon-sample,No,3.00,0.0,3.00,0.00,kRk2WEc602fZs6nPTSkY2lxeV,iPad,Payment,Green Leaf Tea Co.,For Here,00CDETBM9S7YCQSWJ00EY516T8,NaN,ea,1,2018,12
11,2018-12-14,14:45:05,Central Time (US & Canada),Loose tea,Fruit Fiesta,1,Sample,Not available,No,3.00,0.0,3.00,0.00,kRk2WEc602fZs6nPTSkY2lxeV,iPad,Payment,Green Leaf Tea Co.,For Here,00CDETBM9S7YCQSWJ00EY516T8,NaN,ea,1,2018,12


In [ ]:
df2017 = df[df.year == 2017]
df2017.head()

,Date,Time,Time Zone,Category,Item,Quantity,Price Point Name,SKU,Modifiers Applied,Gross Sales,Discounts,Net Sales,Tax,Transaction ID,Device Name,Event Type,Location,Dining Option,Customer ID,Customer Reference ID,Unit,Count,year,month
0,2017-08-30,11:11:23,Central Time (US & Canada),Loose tea,"Green Flower Rooibos, organic",1,4oz,greenflowerrooibos-4,No,11.50,0.0,11.50,0.00,AxVUvEsgClKYFnYsEPqwZsveV,iPad,Payment,Green Leaf Tea Co.,For Here,0078JQACVD65AKGBXWR43K5C6W,NaN,ea,1,2017,8
1,2017-08-30,11:11:23,Central Time (US & Canada),Drinks,Iced Tea,1,Regular,drink-icedtea,No,2.70,0.0,2.70,0.25,AxVUvEsgClKYFnYsEPqwZsveV,iPad,Payment,Green Leaf Tea Co.,For Here,0078JQACVD65AKGBXWR43K5C6W,NaN,ea,1,2017,8
2,2017-12-09,10:35:42,Central Time (US & Canada),Loose tea,Energizing Harmony,1,2 oz,Not available,No,9.00,-1.8,7.20,0.00,6GNSV79jmSWjWO6rCx38oBmeV,iPad,Payment,Green Leaf Tea Co.,For Here,00BHWA7X7D3A0GDA9N2YP5ES2C,NaN,ea,1,2017,12
3,2017-12-09,10:35:42,Central Time (US & Canada),Infusers,Basket Infuser,1,Regular,basketinfuser,No,9.75,0.0,9.75,0.71,6GNSV79jmSWjWO6rCx38oBmeV,iPad,Payment,Green Leaf Tea Co.,For Here,00BHWA7X7D3A0GDA9N2YP5ES2C,NaN,ea,1,2017,12
4,2017-12-09,10:35:42,Central Time (US & Canada),Drinks,Hot Tea,1,Regular,drink-hottea,No,2.70,0.0,2.70,0.24,6GNSV79jmSWjWO6rCx38oBmeV,iPad,Payment,Green Leaf Tea Co.,For Here,00BHWA7X7D3A0GDA9N2YP5ES2C,NaN,ea,1,2017,12


What are the top 5 items sold by year?

In [ ]:
df2019["Item"].value_counts()

Bubble Tea                            2125
Hot Tea                                543
Iced Tea                               426
Custom Amount                          189
Tea Cake                               173
                                      ... 
White Teapot                             1
Halloween High Tea, Sunday Oct 27        1
Regular Tea Of The Month - 3 month       1
Halloween High Tea, Sat Oct 12           1
Green Rooibos                            1
Name: Item, Length: 224, dtype: int64

In [ ]:
df2018["Item"].value_counts()

Bubble Tea             1428
Hot Tea                 341
Iced Tea                277
Custom Amount           218
High Tea                190
                       ... 
Mint Milk Chocolate       1
Decaf Sencha              1
Berry Jubilee             1
Platypus Infuser          1
ACS Bars                  1
Name: Item, Length: 191, dtype: int64

In [ ]:
df2017["Item"].value_counts()

Bubble Tea                          626
Hot Tea                             367
Iced Tea                            222
Custom Amount                       192
Cookie                              170
                                   ... 
Matcha Sticks, Organic                1
Green Tea Sips & Tips                 1
Teahorse Infuser "Under The Tea"      1
Brew Whale Infuser                    1
Mist Iced Tea Jug With Infuser        1
Name: Item, Length: 150, dtype: int64